In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import nltk
from itertools import chain

import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
import re

from nltk.corpus import stopwords
import pickle

from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer\

import string
from datetime import datetime

In [3]:
wordnet_lemmatizer = WordNetLemmatizer()
porter = PorterStemmer()
lancaster=LancasterStemmer()

persongazetteerfilenames = ['lexicon\\people.person.lastnames.modified', 'lexicon\\people.family_name', 'lexicon\\firstname.5000', 'lexicon\\lastname.5000'] 
companygazetteerfilenames = ['lexicon\\business.consumer_company', 'lexicon\\venture_capital.venture_funded_company', 'lexicon\\business.brand']
locationgazetteerfilenames = ['lexicon\\location.country','lexicon\\location','lexicon\\education.university','lexicon\\venues', 'lexicon\\architecture.museum']
productgazetteerfilenames =['lexicon\\product','lexicon\\business.consumer_product']
#,'lexicon\\automotive.model','lexicon\\automotive.make'
titlegazetteerfilenames = ['lexicon\\award.award','lexicon\\base.events.festival_series','lexicon\\book.newspaper', 'lexicon\\tv.tv_program']
groupgazetteerfilenames = ['lexicon\\sports.sports_team']
othergazetteerfilenames = ['lexicon\\time.holiday', 'lexicon\\time.recurring_event']

# other - 'lexicon\\base.events.festival_series','lexicon\\broadcast.tv_channel','lexicon\\cvg.cvg_platform','lexicon\\sports.sports_league', 
#           , 'lexicon\\transportation.road', 'lexicon\\tv.tv_network'

def loadGazetteer(filenames):
    combinedlist = []
    for filename in filenames:
        linelist = [line.rstrip('\n').lower() for line in open(filename, encoding="utf8")]
        #print(len(linelist))
        combinedlist = set().union(combinedlist, linelist)
        #print(len(combinedlist))
    return combinedlist

def isWordInGazette(gazetteer,word):
    if word.lower() in gazetteer:
        return True
    
    return False
    
def isWordGroupInGazette(gazetteer,entity,sentence):
    for entry in gazetteer:
        if entity.lower() in entry:
            if entry in sentence: 
                return True
  
    return False  

"""persongazetteer = loadGazetteer(persongazetteerfilenames)
companygazetteer = loadGazetteer(companygazetteerfilenames)  
locationgazetteer = loadGazetteer(locationgazetteerfilenames)
productgazetteer = loadGazetteer(productgazetteerfilenames)
titlegazetteer = loadGazetteer(titlegazetteerfilenames)
groupgazetteer = loadGazetteer(groupgazetteerfilenames)
othergazetteer = loadGazetteer(othergazetteerfilenames)"""


triggerwordlist = [line.rstrip('\n').lower() for line in open("triggerwordlist.txt", encoding="utf8")]

In [4]:
def isURL(string):
    return len(re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+] |[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', string))>0

def isHashtagUserName(string):
    return (string[0]=='#' or string[0]=='@')

def isAnyDigit(s):
    return any(i.isdigit() for i in s)

def shape(string):
    t1 = re.sub('[A-Z]', 'X',string)
    t2 = re.sub('[a-z]', 'x', t1)
    return re.sub('[0-9]', 'd', t2)

def isAbbr(string):
    regex = re.compile('[A-Z]([A-Z]|\.|&)+') 
    if(regex.search(string) == None):
        return False
    return True

def isPostUpper(post):
    fullpost = ""
    for wordlabel in post:
        word,label = wordlabel
        fullpost += word + " "
    return fullpost.isupper()

def isStopWord(string):
    if string in stopwords.words('english'):
        return True
    return False

def poststring(post, separator=" "):
    fullpost = ""
    for wordlabel in post:
        word,label = wordlabel
        fullpost += word + separator
    return fullpost    

def sentenceTag(post):
    fullpost = ""
    for wordlabel in post:
        word,label = wordlabel
        fullpost += word + " "
    return nltk.pos_tag(fullpost.split())

#gaz_wiki_place = open("gazetteer\\wikipedia_place_titles.pickle", 'rb')
#gaz_wiki_place_db = list(pickle.load(gaz_wiki_place, encoding='bytes'))
#def isPlace(key):
    #for keys in gaz_wiki_place_db:
    #    if key in keys:#
#    if key in gaz_wiki_place_db:
#        return True
#    return False

def isromannum(word):
    validRomanNumerals = ["M", "D", "C", "L", "X", "V", "I"]
    for letters in word.upper():
        if letters not in validRomanNumerals:
            return False
    return True

def haspunctuation(word):
    punctuations = string.punctuation
    for letters in word:
        if letters  in punctuations:
            return True
    return False

def ispunctuation(word):
    punctuations = string.punctuation
    for letters in word:
        if letters not in punctuations:
            return False
    return True

triggerwordlist = [line.rstrip('\n').lower() for line in open("triggerwordlist.txt", encoding="utf8")]
def istriggerword(word):
    if word.lower() in set(triggerwordlist):
        return True
    return False

def wordtypepatterns(poststring):
    pattern =""
    #print("inside ",poststring)
    for word in poststring.split():
        
        if word.islower():
            pattern += "l"
        elif word.isupper():
            pattern += "C"
        elif word.istitle():
            pattern += "T"
        elif word in string.punctuation:
            pattern += "."
        else:
            pattern += "x"

    return pattern

def preprocess(raw_data):
    posts = raw_data.split("\n\n")
    output = []
    for post in posts:
        lines = post.split("\n")
        outputline = []
        for line in lines:
            #print(line)
            if line != "":
                word, label = tuple(line.split("\t"))
                #if(label[0]=='B' or label[0]=='I'):
                #    label = label[2:]
                outputline.append((word,label))
        output.append(outputline)  
    return output 


def preprocessnotag(raw_data):
    posts = raw_data.split("\n\n")
    output = []
    for post in posts:
        
        words = post.split("\n")
        #print(words)
        outputline = []
        for word in words:
            if word != "":
                outputline.append((word," "))
        output.append(outputline)
        #break
    return output 

# REMOVE DUPLICATE POSTS

def removeDuplicate(posts):
    stringlist = []
    result = []
    for post in posts:
        string = poststring(post)
        if string not in stringlist:
            stringlist.append(string)
            result.append(post)
    
    #print(len(posts),len(stringlist))
    return result    


def postPunctuationAsNER(post):
    punctuations = string.punctuation
    for line in post:
        word, label = line
        if word in punctuations and label not in 'O':
            return True
    return False

def removePunctuationAsNER(posts):
    result = []
    
    for post in posts:
        if not postPunctuationAsNER(post):
            result.append(post)
    return result        
        
  



In [5]:
train_raw_data = open("train.txt","r").read()
dev_raw_data = open("dev.txt","r").read()
test_raw_data = open("test_no_tag.txt","r",encoding="utf8").read()

In [6]:
train_posts = preprocess(train_raw_data)
dev_posts = preprocess(dev_raw_data)
test_posts = preprocessnotag(test_raw_data)

train_posts = removeDuplicate(train_posts)   
dev_posts = removeDuplicate(dev_posts)

train_posts =  removePunctuationAsNER(train_posts)  
dev_posts =  removePunctuationAsNER(dev_posts)    


print(len(test_posts))

4271


In [7]:
def word2features(post,i, postag):
    word, label = post[i]
    
    #print(poststring(post))
    sentpattern = wordtypepatterns(poststring(post))
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        #'word[-6:]': word[-6:],
        #'word[-5:]': word[-5:],
        'word[-4:]': word[-4:],
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[:4]': word[:4],
        'word[:3]': word[:3],
        'word[:2]': word[:2],
        'len(word)': len(word),
        'word.islower()': word.islower(),
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.isalpha()': word.isalpha(),
        'word.isalnum()': word.isalnum(),
        'isHashTagUserName(word)':isHashtagUserName(word),
        'istriggerword(word)':istriggerword(word),
       # 'isAnyDigit(word)':isAnyDigit(word),
        'isStopWord(word)':isStopWord(word),
       # 'isAbbr(word)':isAbbr(word),
        'shape(word)':shape(word),
        'isURL(word)':isURL(word),
        'postag': postag[i][1],
        'postag[:2]': postag[i][1][:2],
       # 'isPlace(word)':isPlace(word),
       # 'lemma':wordnet_lemmatizer.lemmatize(word),
       # 'stem.portar':porter.stem(word),
       #'stem.lancaster':lancaster.stem(word),
        'word.isromannum':isromannum(word),
        #'word.haspunctuation':haspunctuation(word),
        'word.ispunctuation':ispunctuation(word),
        
        'isPostUpper(post)':isPostUpper(post),
        #'sentpattern':sentpattern,
    }
    if i > 0:
        word1, label1 = post[i-1]
        features.update({
            '-1:word.islower()': word1.islower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:word.isdigit()': word1.isdigit(),
            '-1:word.isalpha()': word1.isalpha(),
            #'-1:isAnyDigit(word)':isAnyDigit(word1),
            '-1:istriggerword(word)':istriggerword(word1),
            '-1:shape(word)':shape(word1),
            '-1:isURL(word)':isURL(word1),
            '-1:word[-4:]': word1[-4:],
            '-1:word[-3:]': word1[-3:],
            '-1:word[-2:]': word1[-2:],
            '-1:word[:4]': word1[:4],
            '-1:word[:3]': word1[:3],
            '-1:word[:2]': word1[:2],
            '-1:len(word)': len(word1),
            #'-1:isHashTagUserName(word)':isHashtagUserName(word1),
            '-1:postag': postag[i-1][1],
            '-1:postag[:2]': postag[i-1][1][:2]
        })
        if i>1:
            word2, label2 = post[i-2]
            features.update({
                '-2:word.islower()': word2.islower(),
                '-2:word.istitle()': word2.istitle(),
                '-2:word.isupper()': word2.isupper(),
                '-2:word.isdigit()': word2.isdigit(),
                '-2:word.isalpha()': word2.isalpha(),
                '-2:istriggerword(word)':istriggerword(word2),
                '-2:postag': postag[i-2][1],
                '-2:postag[:2]': postag[i-2][1][:2],
                
                
            })
            """if i>2:
                word3, label3 = post[i-3]
                features.update({
                '-3:word.islower()': word3.islower(),
                '-3:word.istitle()': word3.istitle(),
                '-3:word.isupper()': word3.isupper(),
                '-3:word.isdigit()': word3.isdigit(),
                '-3:word.isalpha()': word3.isalpha(),
                '-3:postag': postag[i-3][1],
                '-3:postag[:2]': postag[i-3][1][:2],
                }) """
    else:
        features['BOS'] = True

    if i < len(post)-1:
        word1, label1 = post[i+1]
        features.update({
            '+1:word.islower()': word1.islower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:word.isdigit()': word1.isdigit(),
            '+1:word.isalpha()': word1.isalpha(),
            #'+1:isAnyDigit(word)':isAnyDigit(word1),
            '+1:istriggerword(word)':istriggerword(word1),
            '+1:shape(word)':shape(word1),
            '+1:isURL(word)':isURL(word1),
            '+1:word[-4:]': word1[-4:],
            '+1:word[-3:]': word1[-3:],
            '+1:word[-2:]': word1[-2:],
            '+1:word[:4]': word1[:4],
            '+1:word[:3]': word1[:3],
            '+1:word[:2]': word1[:2],
            '+1:len(word)': len(word1),
            #'+1:isHashTagUserName(word)':isHashtagUserName(word1),
            '+1:postag': postag[i+1][1],
            '+1:postag[:2]': postag[i+1][1][:2]
        })
        if i < len(post) - 2:
            word2, label2 = post[i+2]
            features.update({
                '+2:word.islower()': word2.islower(),
                '+2:word.istitle()': word2.istitle(),
                '+2:word.isupper()': word2.isupper(),
                '+2:word.isdigit()': word2.isdigit(),
                '+2:word.isalpha()': word2.isalpha(),
                '+2:istriggerword(word)':istriggerword(word2),
                '+2:postag': postag[i+2][1],
                '+2:postag[:2]': postag[i+2][1][:2],
            })
            """if i < len(post) - 3:
                word3, label3 = post[i+3]
                features.update({
                '+3:word.islower()': word3.islower(),
                '+3:word.istitle()': word3.istitle(),
                '+3:word.isupper()': word3.isupper(),
                '+3:word.isdigit()': word3.isdigit(),
                '+3:word.isalpha()': word3.isalpha(),
                '+3:postag': postag[i+3][1],
                '+3:postag[:2]': postag[i+3][1][:2],
                })"""
    else:
        features['EOS'] = True

    return features


def post2features(post):
    postag = sentenceTag(post)
    #print("post",post)
    #print("postag in post2features: ",postag)
    return [word2features(post, i, postag) for i in range(len(post))]

def post2labels(post):
    return [label for word, label in post]


In [8]:
%%time
post2features(train_posts[5])[5]

Wall time: 93.8 ms


{'bias': 1.0,
 'word.lower()': 'if',
 'word[-4:]': 'if',
 'word[-3:]': 'if',
 'word[-2:]': 'if',
 'word[:4]': 'if',
 'word[:3]': 'if',
 'word[:2]': 'if',
 'len(word)': 2,
 'word.islower()': True,
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isalpha()': True,
 'word.isalnum()': True,
 'isHashTagUserName(word)': False,
 'istriggerword(word)': False,
 'isStopWord(word)': True,
 'shape(word)': 'xx',
 'isURL(word)': False,
 'postag': 'IN',
 'postag[:2]': 'IN',
 'word.isromannum': False,
 'word.ispunctuation': False,
 'isPostUpper(post)': False,
 '-1:word.islower()': True,
 '-1:word.istitle()': False,
 '-1:word.isupper()': False,
 '-1:word.isdigit()': False,
 '-1:word.isalpha()': True,
 '-1:istriggerword(word)': False,
 '-1:shape(word)': 'xxxx',
 '-1:isURL(word)': False,
 '-1:word[-4:]': 'this',
 '-1:word[-3:]': 'his',
 '-1:word[-2:]': 'is',
 '-1:word[:4]': 'this',
 '-1:word[:3]': 'thi',
 '-1:word[:2]': 'th',
 '-1:len(word)': 4,
 '-1:postag': 'DT',
 '-

In [9]:
%%time
post2labels(train_posts[5])[5]

Wall time: 0 ns


'O'

In [10]:
%%time
X_train = [post2features(s) for s in train_posts]
y_train = [post2labels(s) for s in train_posts]

Wall time: 13.7 s


In [11]:
%%time
X_dev = [post2features(s) for s in dev_posts]
y_dev = [post2labels(s) for s in dev_posts]

Wall time: 5.84 s


In [12]:
%%time
X_test = [post2features(s) for s in test_posts]

Wall time: 16 s


In [13]:
%%time
#0.35000000000000003,0.05
#0.36, 0.08
crf = sklearn_crfsuite.CRF(
    algorithm='pa',
    #c1=0.36,
    #c2=0.08,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

Wall time: 14.6 s


CRF(algorithm='pa', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [14]:
filename = 'crf_lbfgs_'+datetime.now().strftime("%Y%m%d_%H%M")+'.sav'
pickle.dump(crf, open(filename, 'wb'))

In [15]:
labels = list(crf.classes_)
labels.remove('O')
#labels

In [16]:
#0.9397331037451572

y_pred = crf.predict(X_dev)
print("f1 avg ",metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels))
print("f1 micro ",metrics.flat_f1_score(y_dev, y_pred, average='micro', labels=labels))
print("accuracy",metrics.flat_accuracy_score(y_dev, y_pred))

f1 avg  0.2696552104648167
f1 micro  0.30240103828682674
accuracy 0.9428806376114871


In [17]:
test_pred = crf.predict(X_test)

In [18]:
test_posts[1]

[('I', ' '),
 ('want', ' '),
 ('to', ' '),
 ('be', ' '),
 ('the', ' '),
 ('joy', ' '),
 ('to', ' '),
 ('people', ' '),
 ('through', ' '),
 ('my', ' '),
 ('work', ' '),
 ('.', ' '),
 ('-', ' '),
 ('Gianni', ' '),
 ('Versace', ' ')]

In [19]:
test_pred[1]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-person',
 'I-person']

In [20]:
def savepredictions(test_posts, test_pred, filename):
    file = open(filename,"w", encoding="utf8")
    j=0
    for post in test_posts:
        predpostlabel = test_pred[j]
        #print(post)
        for i in range(len(post)):
            word,emptylabel = post[i]
            predlabel = predpostlabel[i]
          #  print(word, predlabel)
            file.write(word+" "+predlabel+"\n")
            
        #if j>2:
         #   break
        file.write("\n")    
        j+=1
    file.close()  
    
    
savepredictions(test_posts, test_pred, filename="test prediction 1_"+datetime.now().strftime("%Y%m%d_%H%M")+".txt")        

In [21]:
"""precision    recall  f1-score   support

      person      0.588     0.429     0.496       266
       title      0.400     0.062     0.108        32
    location      0.616     0.430     0.506       235
     company      0.462     0.122     0.194        49
     product      0.350     0.044     0.079       158
       group      0.333     0.025     0.047       159
       other      0.214     0.183     0.198       229

   micro avg      0.457     0.245     0.319      1128
   macro avg      0.423     0.185     0.232      1128
weighted avg      0.438     0.245     0.292      1128"""
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

   B-company      0.467     0.179     0.259        39
   I-company      0.000     0.000     0.000        10
     B-group      0.273     0.030     0.054       100
     I-group      0.182     0.047     0.074        43
  B-location      0.500     0.418     0.456       141
  I-location      0.417     0.299     0.348        67
     B-other      0.375     0.137     0.201       131
     I-other      0.163     0.075     0.103        93
    B-person      0.512     0.400     0.449       165
    I-person      0.524     0.537     0.530        82
   B-product      0.400     0.129     0.195        31
   I-product      0.250     0.014     0.026        72
     B-title      0.143     0.062     0.087        16
     I-title      0.111     0.083     0.095        12

   micro avg      0.432     0.233     0.302      1002
   macro avg      0.308     0.172     0.206      1002
weighted avg      0.377     0.233     0.270      1002



In [22]:
import seqeval.metrics  as seqevalmetrics
seqevalmetrics.f1_score(y_dev, y_pred)

0.2957600827300931

In [23]:
seqevalmetrics.accuracy_score(y_dev, y_pred)

0.9428806376114871

In [24]:
print(seqevalmetrics.classification_report(y_dev, y_pred, digits=3))

           precision    recall  f1-score   support

    other      0.312     0.115     0.168       131
    group      0.250     0.030     0.054       100
 location      0.445     0.376     0.408       141
   person      0.485     0.382     0.427       165
  company      0.375     0.154     0.218        39
    title      0.125     0.062     0.083        16
  product      0.182     0.065     0.095        31

micro avg      0.416     0.230     0.296       623
macro avg      0.371     0.230     0.270       623



In [52]:
seqevalmetrics.classification_report(y_dev, y_pred)

'           precision    recall  f1-score   support\n\n   person       0.47      0.35      0.40       165\n location       0.56      0.36      0.44       141\n    group       0.33      0.02      0.04       100\n    title       0.00      0.00      0.00        16\n    other       0.28      0.14      0.18       131\n  company       0.64      0.18      0.28        39\n  product       0.33      0.06      0.11        31\n\nmicro avg       0.46      0.22      0.30       623\nmacro avg       0.42      0.22      0.27       623\n'

In [30]:
import numpy
#c1list =  numpy.append((numpy.arange(0.05, 0.51, 0.05)),[1,2,4,7,10])
c1list =  numpy.arange(0.3, 0.41, 0.01)
#c2list = numpy.append((numpy.arange(0.05, 0.51, 0.05)),[1,2,4,7,10])
c2list =  numpy.arange(0.01, 0.11, 0.01)

#0.35000000000000003,0.05
print(c1list)
print(c2list)

[0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37 0.38 0.39 0.4 ]
[0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1 ]


In [31]:
%%time
maxaccuracy = 0
maxf1avg = 0
maxf1micro = 0

result = []
for c1val in c1list:
    for c2val in c2list:
        crf = sklearn_crfsuite.CRF(
            algorithm='lbfgs',
            c1=c1val,
            c2=c2val,
            max_iterations=100,
            all_possible_transitions=False
        )
        crf.fit(X_train, y_train)
        
        labels = list(crf.classes_)
        labels.remove('O')
        
        y_pred = crf.predict(X_dev)
        
        f1avg = metrics.flat_f1_score(y_dev, y_pred, average='weighted', labels=labels)
        f1micro = metrics.flat_f1_score(y_dev, y_pred, average='micro', labels=labels)
        accuracy = metrics.flat_accuracy_score(y_dev, y_pred)
        
        if(f1avg>maxf1avg):
            maxf1avg = f1avg
        if(f1micro>maxf1micro):
            maxf1micro = f1micro
        if(accuracy>maxaccuracy):
            maxaccuracy = accuracy
        
        print("f1 avg ",f1avg)
        print("f1 micro ",f1micro)
        print("accuracy",accuracy)
        
        tempresult = c1val, c2val, f1avg, f1micro, accuracy
        result.append(tempresult)
        
        print(datetime.now().strftime("%Y%m%d_%H%M")+" completed ("+str(c1val)+","+str(c2val)+")")

        
print("max f1avg", maxf1avg, ", max f1micro", maxf1micro, ", max accuracy,", maxaccuracy)        
filename = 'crf_compare_'+datetime.now().strftime("%Y%m%d_%H%M")+'.pkl'
pickle.dump(result, open(filename, 'wb'))

f1 avg  0.27960768069322056
f1 micro  0.2981818181818182
accuracy 0.9392118413561895
20200208_0748 completed (0.3,0.01)
f1 avg  0.27710891467100346
f1 micro  0.2953020134228188
accuracy 0.9395913720032892
20200208_0749 completed (0.3,0.02)
f1 avg  0.2828630929320414
f1 micro  0.3007518796992481
accuracy 0.9411727496995382
20200208_0750 completed (0.3,0.03)
f1 avg  0.27652267259519897
f1 micro  0.29528535980148884
accuracy 0.9404136884053387
20200208_0750 completed (0.3,0.04)
f1 avg  0.2679076934122152
f1 micro  0.2835913312693498
accuracy 0.9395281168954394
20200208_0751 completed (0.3,0.05)
f1 avg  0.2823164656384054
f1 micro  0.3036053130929791
accuracy 0.9416787905623379
20200208_0751 completed (0.3,0.060000000000000005)
f1 avg  0.25439753971222917
f1 micro  0.27616645649432536
accuracy 0.9396546271111392
20200208_0752 completed (0.3,0.06999999999999999)
f1 avg  0.27470162568141654
f1 micro  0.29479034307496826
accuracy 0.941552280346638
20200208_0753 completed (0.3,0.08)
f1 avg  0.

In [32]:
print(maxaccuracy, maxf1avg, maxf1micro)

0.9435764437978367 0.295614939444403 0.319693094629156


In [2]:
from collections import Counter
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:


NameError: name 'crf' is not defined

In [ ]:
#loaded_model = pickle.load(open("crf_best_estimator_3_fold_20200201_0055.sav", 'rb'))
#result = loaded_model.score(X_dev, y_dev)
#print(result)

In [ ]:
#loaded_model = pickle.load(open("crf_lbfgs_i100.sav", 'rb'))
#result = loaded_model.score(X_dev, y_dev)
#print(result)

In [25]:
import eli5
eli5.show_weights(crf, top=30)

Using TensorFlow backend.


From \ To,O,B-company,I-company,B-group,I-group,B-location,I-location,B-other,I-other,B-person,I-person,B-product,I-product,B-title,I-title
O,0.324,0.179,-0.12,0.187,-0.199,0.154,-0.25,0.138,-0.268,0.255,-0.184,0.164,-0.147,0.165,-0.193
B-company,0.054,-0.126,0.279,-0.014,-0.031,0.045,-0.031,-0.011,-0.043,-0.04,-0.05,-0.015,-0.04,-0.011,-0.026
I-company,0.006,-0.021,0.112,-0.005,-0.004,-0.008,-0.02,-0.012,-0.014,-0.004,-0.01,-0.006,-0.01,0.0,-0.004
B-group,-0.004,-0.014,-0.01,-0.084,0.394,-0.011,-0.035,-0.019,-0.051,-0.022,-0.032,-0.005,-0.011,-0.01,-0.017
I-group,-0.04,-0.014,-0.019,-0.031,0.197,-0.025,-0.017,-0.008,-0.033,-0.015,-0.017,-0.01,-0.019,-0.001,-0.005
B-location,0.038,-0.028,-0.045,-0.031,-0.065,-0.067,0.419,-0.029,-0.1,-0.031,-0.061,-0.027,-0.049,-0.016,-0.041
I-location,-0.034,-0.008,-0.023,0.027,-0.022,-0.037,0.214,-0.011,-0.044,0.006,-0.047,-0.015,-0.037,-0.005,-0.038
B-other,-0.064,-0.021,-0.023,-0.013,-0.038,-0.016,-0.036,-0.075,0.483,0.003,-0.03,-0.013,-0.028,-0.005,-0.038
I-other,-0.03,-0.024,-0.035,-0.021,-0.042,-0.044,-0.05,-0.013,0.387,-0.001,-0.036,-0.031,-0.02,-0.009,-0.038
B-person,0.044,-0.023,-0.042,-0.021,-0.049,-0.021,-0.075,-0.028,-0.079,-0.144,0.48,-0.024,-0.036,-0.008,-0.052


In [26]:
eli5.show_weights(crf, top=(10,10), feature_re='^word\.is',
                  horizontal_layout=False, show=['targets'])

Weight?,Feature
+0.422,word.ispunctuation
+0.297,word.islower()
+0.081,word.isdigit()
+0.042,word.isromannum
+0.013,word.isupper()
-0.037,word.isalpha()
-0.116,word.istitle()
-0.160,word.isalnum()
Weight?,Feature
+0.040,word.islower()


In [27]:
eli5.show_weights(crf, top=(10,10), feature_re='.lower.',
                  horizontal_layout=False, show=['targets'])

Weight?,Feature
+0.297,word.islower()
+0.116,word.lower():tonight
+0.105,word.lower():go
+0.101,word.lower():lol
+0.095,word.lower():rt
+0.093,word.lower():someone
+0.087,word.lower():free
+0.087,word.lower():scm
+0.086,word.lower():doctor
+0.081,word.lower():prison


In [ ]:
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation //flat_f1_score, average='weighted', labels=labels
f1_scorer = make_scorer(metrics.flat_accuracy_score)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose=1, 
                        n_jobs=-1, 
                        n_iter=50, 
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

In [ ]:
#crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
filename = 'crf_best_estimator_3_fold_'+datetime.now().strftime("%Y%m%d_%H%M")+'.sav'
pickle.dump(rs.best_estimator_, open(filename, 'wb'))

In [ ]:
crf_best = rs.best_estimator_
y_pred = crf_best.predict(X_dev)
print(metrics.flat_classification_report(
    y_dev, y_pred, labels=sorted_labels, digits=3
))

In [ ]:
seqevalmetrics.f1_score(y_dev, y_pred)

In [ ]:
seqevalmetrics.accuracy_score(y_dev, y_pred)

In [ ]:
print(seqevalmetrics.classification_report(y_dev, y_pred, digits=3))

In [ ]:
test_pred_best = crf_best.predict(X_test)
savepredictions(test_posts, test_pred_best, filename="test prediction 1_best_"+datetime.now().strftime("%Y%m%d_%H%M")+".txt")    

In [ ]:
def comparepredictions(pred1, pred2):
    
    #print(pred1[:2])
    
    match = True
    
    if(len(pred1) != len(pred2)):
        print("error")
        return
    
    for i in range(len(pred1)):
        for j in range(len(pred1[i])):
            pred1result = pred2[i][j]
            pred2result = pred2[i][j]
            if pred1result != pred2result:
                print(pred1result,pred2result)
                match = False
       
    if(match):
        print("All same")
        
comparepredictions(test_pred, test_pred_best)

In [ ]:
from sklearn.model_selection import cross_val_predict
crosspred = cross_val_predict(estimator=crf, X=X_train, y=y_train, cv=5)

In [ ]:
print(seqevalmetrics.classification_report(y_dev, crosspred, digits=3))

In [ ]:
crosspred[0]

In [ ]:
loaded_model = pickle.load(open("crf_best_estimator_3_fold_20200207_1221.pkl", 'rb'))
result = loaded_model.score(X_dev, y_dev)
print(result)